# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [ ]:
%pip install rdkit torchmetrics ml-collections polars clu -q
%pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
%pip install pcsaft -q

In [ ]:
%pip install -U "ray[tune]" protobuf==3.19 tensorboardx -q
%pip install hpbandster ConfigSpace -q

In [ ]:
%pip show pcsaft | grep -E "Version|Name"

In [ ]:
!nvidia-smi

In [ ]:
!lscpu | grep -E "Model name|Core|core" 

## PNA EPCSAFT

In [ ]:
!python -m train.tuner --workdir=/notebooks/GNN-ePC-SAFT --config=configs/default.py --dataset=ramirez\
--num_cpu=8 --num_gpus=1 --num_samples=50 --get_result=False\
--config.num_train_steps=11000 --config.log_every_steps=1000 \
--config.num_para=3 --restoredir=/notebooks/GNN-ePC-SAFT/ray/gnnpcsaft

In [ ]:
import wandb
wandb.login()

In [ ]:
!python -m train.train --workdir=./ --config=configs/default.py --dataset=thermoml\
--config.num_train_steps=5_210_000 --config.optimizer=adam --config.learning_rate=1e-3 \
--config.change_opt=False --config.change_sch=True \
--config.log_every_steps=5_000 --config.eval_every_steps=10_000 --config.checkpoint_every_steps=10_000 \
--config.propagation_depth=4 --config.hidden_dim=128 --config.num_mlp_layers=2 --config.pre_layers=1 --config.post_layers=3

In [ ]:
for n in range(1,10,1):
    gamma = 1*10**-n
    !python -m train.parametrisation --weight_decay={gamma}

In [ ]:
!python -m train.parametrisation --weight_decay=0.0